# 1. Importing modules and functions

In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
import chembl_structure_pipeline
from molvs import standardize_smiles
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import joblib
import pickle
from numpy import savetxt
from padelpy import from_sdf
from IPython.display import HTML
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
import warnings
warnings.filterwarnings('ignore')

[02:30:31] Initializing Normalizer


# 2.Data entry and curation work set

In [2]:
uploaded_file_ws="datasets/HDAC3_work.sdf"
supplier_ws = Chem.ForwardSDMolSupplier(uploaded_file_ws,sanitize=False)
failed_mols_ws = []
all_mols_ws =[]
wrong_structure_ws=[]
wrong_smiles_ws=[]
y_tr = []
y_bad_index=[]

for i, m in enumerate(supplier_ws):
    structure = Chem.Mol(m)
    all_mols_ws.append(structure)
    y_tr.append(m.GetProp("pchembl_value_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ws.append(m)
        wrong_smiles_ws.append(Chem.MolToSmiles(m))
        wrong_structure_ws.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ws), 'molecules')
print('Failed data: ', len(failed_mols_ws), 'molecules')
number_ws =[]
for i in range(len(failed_mols_ws)):
        number_ws.append(str(i+1))
bad_molecules_ws = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ws, 'SMILES of wrong structure: ': wrong_smiles_ws, 'No.': number_ws}, index=None)
bad_molecules_ws = bad_molecules_ws.set_index('No.')
bad_molecules_ws

Original data:  1400 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [3]:
y_tr[:] = [x for i,x in enumerate(y_tr) if i not in y_bad_index]

In [4]:
len(y_tr)

1400

# 3.Standardization SDF file for work set

In [5]:
all_mols_ws[:] = [x for i,x in enumerate(all_mols_ws) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ws)):
    record = Chem.MolToSmiles(all_mols_ws[i])
    records.append(record)

moldf_ws = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ws.append(m)
    
print('Kept data: ', len(moldf_ws), 'molecules')

Kept data:  1400 molecules


In [6]:
moldf_ws=pd.DataFrame(moldf_ws, columns=['Mol'])
moldf_ws

,Mol
0,<rdkit.Chem.rdchem.Mol object at 0x000001BD7D1...
1,<rdkit.Chem.rdchem.Mol object at 0x000001BD7B0...
2,<rdkit.Chem.rdchem.Mol object at 0x000001BD7D1...
3,<rdkit.Chem.rdchem.Mol object at 0x000001BD7D1...
4,<rdkit.Chem.rdchem.Mol object at 0x000001BD7D1...
...,...
1395,<rdkit.Chem.rdchem.Mol object at 0x000001BD7D1...
1396,<rdkit.Chem.rdchem.Mol object at 0x000001BD7D1...
1397,<rdkit.Chem.rdchem.Mol object at 0x000001BD7D1...
1398,<rdkit.Chem.rdchem.Mol object at 0x000001BD7D1...


# 4.Data entry and curation test set

In [7]:
uploaded_file_ts="datasets/HDAC3_test.sdf"
supplier_ts = Chem.ForwardSDMolSupplier(uploaded_file_ts,sanitize=False)
failed_mols_ts = []
all_mols_ts =[]
wrong_structure_ts=[]
wrong_smiles_ts=[]
y_ts = []
y_bad_index=[]
for i, m in enumerate(supplier_ts):
    structure = Chem.Mol(m)
    all_mols_ts.append(structure)
    y_ts.append(m.GetProp("pchembl_value_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ts.append(m)
        wrong_smiles_ts.append(Chem.MolToSmiles(m))
        wrong_structure_ts.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ts), 'molecules')
print('Failed data: ', len(failed_mols_ts), 'molecules')
number_ts =[]
for i in range(len(failed_mols_ts)):
        number_ts.append(str(i+1))
bad_molecules_ts = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ts, 'SMILES of wrong structure: ': wrong_smiles_ts, 'No.': number_ts}, index=None)
bad_molecules_ts = bad_molecules_ts.set_index('No.')
bad_molecules_ts

Original data:  351 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [8]:
y_ts[:] = [x for i,x in enumerate(y_ts) if i not in y_bad_index]

In [9]:
len(y_ts)

351

# 5.Standardization SDF file for test set

In [10]:
all_mols_ts[:] = [x for i,x in enumerate(all_mols_ts) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ts)):
    record = Chem.MolToSmiles(all_mols_ts[i])
    records.append(record)

moldf_ts = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ts.append(m)
    
print('Kept data: ', len(moldf_ts), 'molecules')

Kept data:  351 molecules


In [150]:
moldf_ts=pd.DataFrame(moldf_ts, columns=['Mol'])
moldf_ts

,Mol
0,<rdkit.Chem.rdchem.Mol object at 0x0000019272B...
1,<rdkit.Chem.rdchem.Mol object at 0x0000019272B...
2,<rdkit.Chem.rdchem.Mol object at 0x0000019272B...
3,<rdkit.Chem.rdchem.Mol object at 0x0000019272B...
4,<rdkit.Chem.rdchem.Mol object at 0x0000019272B...
...,...
346,<rdkit.Chem.rdchem.Mol object at 0x00000192048...
347,<rdkit.Chem.rdchem.Mol object at 0x00000192048...
348,<rdkit.Chem.rdchem.Mol object at 0x00000192048...
349,<rdkit.Chem.rdchem.Mol object at 0x00000192048...


In [13]:
from pathlib import Path

In [14]:
path = Path('feature_name_rfecv_MF.txt')
feature_name_rfecv_MF = path.read_text().splitlines()

# 6.Calculation MorganFingerprint for work set

In [11]:
def calcfp(mol,funcFPInfo=dict(radius=2, nBits=1024, useFeatures=False, useChirality=False)):
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, **funcFPInfo)
    fp = pd.Series(np.asarray(fp))
    fp = fp.add_prefix('Bit_')
    return fp

# Training set
desc_ws = moldf_ws.Mol.apply(calcfp)
desc_ws

,Bit_0,Bit_1,Bit_2,Bit_3,Bit_4,Bit_5,Bit_6,Bit_7,Bit_8,Bit_9,...,Bit_1014,Bit_1015,Bit_1016,Bit_1017,Bit_1018,Bit_1019,Bit_1020,Bit_1021,Bit_1022,Bit_1023
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
1396,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
1397,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
1398,0,1,0,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [16]:
x_tr=desc_ws[feature_name_rfecv_MF]
x_tr

,Bit_1,Bit_4,Bit_8,Bit_15,Bit_25,Bit_31,Bit_33,Bit_36,Bit_41,Bit_42,...,Bit_961,Bit_967,Bit_980,Bit_997,Bit_999,Bit_1009,Bit_1010,Bit_1016,Bit_1017,Bit_1019
0,1,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,1,0,0,0,0,0,1,1,0,0,...,0,0,0,1,0,0,1,1,0,1
1396,1,0,0,0,0,0,1,1,0,0,...,0,0,0,1,0,0,1,1,0,1
1397,1,0,0,0,0,0,1,1,0,0,...,0,0,0,1,0,0,1,1,0,1
1398,1,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [20]:
x_tr.to_numpy()

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 1, 0, 1],
       [1, 1, 1, ..., 0, 0, 1],
       [1, 0, 0, ..., 1, 0, 1]])

In [22]:
np.savetxt("data3.csv", x_tr,  
              delimiter = ",")

In [19]:
x_tr.reset_index().to_csv("x_tr_MF.csv", index=False)

In [155]:
y_tr = np.array(y_tr, dtype=np.float32)
len(y_tr)

1400

# 7.Calculation MorganFingerprint for test set

In [156]:
desc_ts = moldf_ts.Mol.apply(calcfp)
desc_ts

,Bit_0,Bit_1,Bit_2,Bit_3,Bit_4,Bit_5,Bit_6,Bit_7,Bit_8,Bit_9,...,Bit_1014,Bit_1015,Bit_1016,Bit_1017,Bit_1018,Bit_1019,Bit_1020,Bit_1021,Bit_1022,Bit_1023
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
347,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
348,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
349,0,0,1,0,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0


In [157]:
y_ts = np.array(y_ts, dtype=np.float32)

In [158]:
x_ts=desc_ts[feature_name_rfecv_MF]

In [159]:
x_ts

,Bit_1,Bit_4,Bit_8,Bit_15,Bit_25,Bit_31,Bit_33,Bit_36,Bit_41,Bit_42,...,Bit_961,Bit_967,Bit_980,Bit_997,Bit_999,Bit_1009,Bit_1010,Bit_1016,Bit_1017,Bit_1019
0,1,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,1,0,...,0,0,1,0,1,0,0,0,0,1
3,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1,0,0,0,0,0,1,1,0,0,...,0,0,0,1,0,0,1,1,0,1
347,1,0,0,0,0,0,1,1,0,0,...,1,0,0,0,0,0,1,1,0,1
348,1,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,1,1,0,1
349,0,1,0,0,0,1,1,1,0,0,...,0,0,0,1,0,0,0,1,0,1


# load the models from disk

In [25]:
best_svm = pickle.load(open('Models/Morgan_FP/HDAC3_SVM_Morgan_FP.pkl', 'rb'))

In [26]:
best_gbr = pickle.load(open('Models/Morgan_FP/HDAC3_GBR_MF_final_FS.pkl', 'rb'))

# Prediction for CV

In [116]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [160]:
y_pred_CV_svm = cross_val_predict(best_svm, x_tr, y_tr, cv=cv)

In [161]:
y_pred_CV_gbr = cross_val_predict(best_gbr, x_tr, y_tr, cv=cv)

In [162]:
y_pred_con=(y_pred_CV_svm+y_pred_CV_gbr)/2

In [163]:
Q2_CV = round(r2_score(y_tr, y_pred_con), 2)
Q2_CV

0.7

In [164]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_con)),2)
RMSE_CV

0.67

# Prediction for test set's molecules

In [171]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [172]:
y_pred_svm = best_svm.predict(x_ts)

In [173]:
y_pred_gbr = best_gbr.predict(x_ts)

In [174]:
y_pred_con=(y_pred_svm+y_pred_gbr)/2

In [175]:
Q2_TS = round(r2_score(y_ts, y_pred_con), 2)
Q2_TS

0.69

In [176]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts, y_pred_con)), 2)
RMSE_TS

0.69

# Estimating applicability domain. Method - Euclidian distances, K=1

In [177]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [178]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1.732051,2.645751,2.645751,2.449490,2.449490,3.162278,2.236068,3.741657,4.000000,3.162278,...,3.162278,2.449490,2.449490,1.000000,1.414214,2.236068,2.236068,2.000000,4.358899,3.162278
2,2.000000,2.828427,3.162278,2.449490,2.828427,3.464102,3.872983,3.741657,4.358899,3.316625,...,4.123106,3.162278,2.828427,3.000000,1.732051,2.236068,2.645751,2.000000,4.795832,3.316625
3,2.000000,2.828427,3.605551,2.828427,4.123106,3.741657,3.872983,3.872983,4.472136,3.316625,...,4.242641,3.316625,2.828427,3.000000,2.449490,2.236068,3.162278,2.000000,5.000000,3.605551
4,2.236068,3.162278,4.123106,3.000000,4.242641,3.872983,4.000000,4.242641,4.582576,4.123106,...,4.358899,3.605551,3.000000,3.162278,2.449490,2.236068,3.316625,2.236068,5.099020,3.605551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,7.745967,7.937254,8.544004,7.874008,7.810250,8.485281,8.306624,7.745967,8.831761,8.306624,...,8.426150,8.426150,8.888194,8.831761,8.831761,9.055385,9.000000,9.110434,8.185353,9.000000
1396,7.810250,8.000000,8.602325,7.874008,7.874008,8.485281,8.366600,7.810250,8.831761,8.306624,...,8.485281,8.485281,8.944272,8.831761,8.888194,9.055385,9.000000,9.110434,8.185353,9.000000
1397,7.810250,8.000000,8.717798,7.937254,7.937254,8.485281,8.426150,7.810250,8.831761,8.306624,...,8.544004,8.544004,8.944272,8.888194,8.888194,9.110434,9.000000,9.110434,8.246211,9.000000
1398,7.874008,8.062258,8.717798,7.937254,8.000000,8.485281,8.485281,7.874008,8.831761,8.366600,...,8.602325,8.544004,9.000000,8.944272,9.000000,9.165151,9.055385,9.165151,8.366600,9.055385


In [179]:
similarity= neighbors_k

In [180]:
Dmean=np.mean(similarity[1,:])

In [181]:
round(Dmean, 2)

2.24

In [182]:
std=np.std(similarity[1,:])

In [183]:
round(std, 2)

1.08

In [184]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

2.78


In [185]:
x_tr.shape

(1400, 191)

In [186]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [187]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,341,342,343,344,345,346,347,348,349,350
0,0.000000,3.000000,5.385165,3.741657,2.236068,2.645751,2.449490,1.414214,3.464102,2.645751,...,1.732051,2.449490,2.236068,2.449490,2.449490,2.645751,2.236068,2.645751,3.605551,1.000000
1,2.236068,3.741657,5.656854,3.741657,2.828427,3.000000,4.000000,1.732051,3.605551,2.828427,...,2.000000,2.645751,2.449490,3.162278,2.828427,3.000000,2.449490,2.828427,3.605551,1.000000
2,2.236068,4.242641,5.656854,3.872983,3.162278,3.000000,4.242641,2.449490,3.741657,3.000000,...,2.449490,3.316625,2.828427,3.162278,3.162278,3.162278,3.000000,2.828427,3.741657,1.414214
3,2.236068,4.242641,5.656854,4.242641,3.162278,3.464102,4.358899,2.449490,4.795832,3.316625,...,2.449490,3.316625,2.828427,3.162278,3.162278,3.162278,3.000000,3.000000,3.872983,2.236068
4,2.645751,4.358899,5.744563,4.358899,4.123106,3.605551,4.582576,2.828427,4.898979,3.605551,...,2.645751,3.464102,3.000000,3.316625,3.316625,3.162278,3.316625,3.162278,4.123106,3.316625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,7.810250,7.874008,8.602325,9.110434,7.810250,8.000000,8.944272,7.874008,8.485281,8.000000,...,8.306624,8.246211,8.306624,9.165151,8.774964,9.219544,8.831761,8.831761,8.485281,8.306624
1396,7.810250,7.874008,8.602325,9.110434,7.810250,8.062258,8.944272,7.874008,8.485281,8.062258,...,8.306624,8.306624,8.366600,9.165151,8.831761,9.273618,8.888194,8.944272,8.485281,8.366600
1397,7.874008,8.000000,8.602325,9.219544,7.810250,8.185353,8.944272,8.000000,8.485281,8.185353,...,8.306624,8.306624,8.366600,9.219544,8.831761,9.273618,8.888194,8.944272,8.485281,8.366600
1398,7.874008,8.000000,8.602325,9.273618,7.874008,8.185353,9.000000,8.000000,8.602325,8.185353,...,8.306624,8.306624,8.366600,9.327379,8.888194,9.327379,8.944272,8.944272,8.485281,8.485281


In [188]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[0.    3.    5.385 3.742 2.236 2.646 2.449 1.414 3.464 2.646 3.162 2.646
 3.    2.828 3.    1.    1.    1.414 3.742 3.    2.236 3.742 3.    0.
 3.    1.    1.732 2.236 2.646 2.236 3.317 2.    3.606 3.317 2.828 0.
 3.162 2.236 2.449 2.236 2.828 3.464 3.464 3.742 2.646 3.464 3.606 3.
 1.    1.414 1.732 3.464 3.162 2.236 1.414 3.606 1.414 2.828 2.828 2.828
 2.    3.    2.828 3.873 3.873 1.732 4.    2.646 1.    4.123 3.742 2.449
 2.236 2.828 0.    0.    2.    2.449 3.    3.464 2.236 2.236 2.    2.828
 2.646 1.414 2.449 1.732 2.    2.828 2.449 4.123 3.873 2.    0.    3.317
 1.414 2.    0.    2.    4.123 1.414 2.449 0.    2.    2.646 3.    0.
 2.646 1.    2.236 1.732 1.414 2.236 4.    1.    2.    0.    2.449 2.
 3.606 1.732 3.    3.    3.162 2.449 3.162 2.449 1.732 3.742 1.414 2.236
 1.414 3.    1.414 1.732 2.    3.162 3.606 1.    4.583 1.    2.646 1.732
 2.236 2.646 1.414 2.646 2.    4.472 1.    3.162 2.646 1.732 0.    1.732
 1.732 1.    3.    2.236 1.    2.449 1.732 2.646 0.    1.732 3.   

In [189]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True False False False  True  True  True  True False  True False  True
 False False False  True  True  True False False  True False False  True
 False  True  True  True  True  True False  True False False False  True
 False  True  True  True False False False False  True False False False
  True  True  True False False  True  True False  True False False False
  True False False False False  True False  True  True False False  True
  True False  True  True  True  True False False  True  True  True False
  True  True  True  True  True False  True False False  True  True False
  True  True  True  True False  True  True  True  True  True False  True
  True  True  True  True  True  True False  True  True  True  True  True
 False  True False False False  True False  True  True False  True  True
  True False  True  True  True False False  True False  True  True  True
  True  True  True  True  True False  True False  True  True  True  True
  True  True False  True  True  True  True  True  T

In [190]:
print("Coverage = ", sum(cpd_AD) / len(cpd_AD))

Coverage =  0.6752136752136753


In [191]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  0   4   5   6   7   9  11  15  16  17  20  23  25  26  27  28  29  31
  35  37  38  39  44  48  49  50  53  54  56  60  65  67  68  71  72  74
  75  76  77  80  81  82  84  85  86  87  88  90  93  94  96  97  98  99
 101 102 103 104 105 107 108 109 110 111 112 113 115 116 117 118 119 121
 125 127 128 130 131 132 134 135 136 139 141 142 143 144 145 146 147 148
 150 152 153 154 155 156 157 159 160 161 162 163 164 165 167 168 169 170
 171 172 173 174 176 178 180 183 184 185 187 188 190 191 192 193 195 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 220 221 222 223 226 227 228 230 232 233 237 238 239 240 242
 244 246 247 248 250 251 252 253 254 255 256 257 258 259 260 263 264 265
 266 267 268 269 270 272 275 277 281 282 283 284 286 289 290 291 292 294
 299 300 301 303 304 306 307 311 312 313 315 316 317 318 319 321 323 324
 325 326 327 329 331 332 333 334 335 336 338 339 341 342 343 344 345 346
 347 348 35

In [192]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# Prediction only for molecules included in  AD

In [193]:
y_pred_con_ad=list(y_pred_con)

In [194]:
y_pred_con_ad[:] = [x for i,x in enumerate(y_pred_con_ad) if i not in out_Ad]

In [195]:
len(y_pred_con_ad)

237

In [196]:
y_ts_ad=list(y_ts)

In [197]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [198]:
len(y_ts_ad)

237

In [199]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_con_ad), 2)
Q2_TS

0.75

In [200]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts_ad, y_pred_con_ad)), 2)
RMSE_TS

0.6